# CSE 601 Data Mining and Bioinformatics
# Project 3: Classification Algorithms

# Part 1: K-Nearest Neighbor
Description: This program conducts KNN Classification Algorithm on the given datasets.
    
Submitted By:
    
Name: Karan Manchandia | 
UBIT: karanman | 
Person No.: 50290755
    
Name: Divya Srivastava | 
UBIT: divyasri | 
Person No.: 50290383

Name: Varsha Lakshman | 
UBIT: varshala | 
Person No.: 50288138 

In [1]:
# Importing Libraries
from scipy.spatial import distance
import operator
import numpy as numpy
import math

In [2]:
# This function divides the data as per the number of cross validation folds into training and testing data
# This function then calls the knn algorithm function
def knn_cvfold(cv_folds, processed_data, K):
    # Defining four lists in which performance parameters of folds will be stored
    k_accuracy = []
    k_precision = []
    k_recall = []
    k_fmeasure = []
    
    # Calculating total records per fold
    record_per_fold_float = (len(processed_data)/cv_fold)
    record_per_fold = math.floor(record_per_fold_float) + 1
    
    # Generating the training and testing datasets and running the KNN algorithm
    kth_fold = 1
    while kth_fold < cv_fold + 1:
        # Dividing the data in train and test sets.
        processed_data_1 = list(processed_data)
        processed_test_data = processed_data_1[(kth_fold - 1) * record_per_fold : kth_fold * record_per_fold]
        del processed_data_1[(kth_fold - 1) * record_per_fold : kth_fold * record_per_fold - 1]
        processed_train_data = processed_data_1
        A, P, R, FM = knn_algorithm(processed_train_data, processed_test_data, K)
        # Appending performance measures to their respective lists 
        k_accuracy.append(A)
        k_precision.append(P)
        k_recall.append(R)
        k_fmeasure.append(FM)
        kth_fold += 1
    
    # Function to calculate average of perfromance measures list
    def Average(lst): 
        return sum(lst) / len(lst) 
    
    # Printing Accuracy, Precision, Recall and f-measure
    print(f"Accuracy = {Average(k_accuracy)}")
    print(f"Precision = {Average(k_precision)}")
    print(f"Recall = {Average(k_recall)}")
    print(f"F-measure = {Average(k_fmeasure)}")

In [3]:
# This function calculates accuracy, precision, recall and f-measure for the model classifier

def aprf_performance (given_classes, estimated_classes):
    # Defining four variables
    # a(true positive): The case when actual class is "Yes" and predicted class is also "Yes"
    # b(false negative): The case when actual class is "Yes" and predicted class is "No"
    # c(flase positive): The case when actual class is "No" and predicted class is "Yes"
    # d(true nrgative): The case when actual class is "No" and predicted class is also "No"
    # Yes/ No corresponds to 1/0.
    a = 0
    b = 0
    c = 0
    d = 0
    
    # defining a new variable
    # Calculating values of a,b,c,d.
    temp15 = 0
    while temp15 < len(given_classes):
        if(given_classes[temp15] == 1.0 and estimated_classes[temp15][-1] == 1.0):
            a += 1
        elif(given_classes[temp15] == 0.0 and estimated_classes[temp15][-1] == 1.0):
            c += 1
        elif(given_classes[temp15] == 0.0 and estimated_classes[temp15][-1] == 0.0):
            d += 1
        else:
            b += 1
        temp15 += 1
        
    # Correctly classified records
    A_num = a+d
    # Total Records
    A_den = len(given_classes)
    # Calculating Accuracy (A)
    A = A_num/A_den
    P_den = a+c
    # Calculating precision (P)
    P = a/P_den 
    R_den = a+b
    # Calculating recall (R)
    R = a/R_den
    fm_num = 2*a
    fm_den = (2*a)+b+c
    # Calculating f-measure
    FM = fm_num/fm_den
    
    return (A, P, R, FM)

In [4]:
# This function is responsible for reading the datafile, creating a list of list where each list is a row
# This function also take care of float and string values in the datafile. The string values are assigned appropriate integers.
def data_read_process(my_file):
    
    # Defining an empty list
    processed_data = []
    
    # Opening file and as data__file
    with open(my_file) as data__file:
        records = []
        
        # Creating a list of datafile which contains list of all records
        for item in data__file:
            temp1 = item.replace("\n","")
            temp2 = temp1.split("\t")
            records.append(temp2)
        
        # Taking care of the float values in the dataset
        for row in records:
            temp5 = []
            for value in row:
                def check_f(value1):
                    try:
                        temp_t = float(value1)
                        return True
                    except ValueError:
                        return False
                if check_f(value):
                    temp5.append(float(value))
                else:
                    temp5.append(value)
            processed_data.append(temp5)

        # Taking care of strings in the dataset(assigning integers for strings)
        q = 0
        while q < len(processed_data[0]):
            if isinstance(processed_data[0][q],str):
                temp11 = []
                for row1 in processed_data:
                    temp10 = row1[q]
                    temp11.append(temp10)
                    strings_in_dox = set(temp11)
                iter_strings = 0
                str_int_maping = {}
                for item2 in strings_in_dox:
                    if item2 in str_int_maping:
                        continue
                    str_int_maping[item2] = iter_strings
                    iter_strings += 1
                for row1 in processed_data:
                    row1[q] = str_int_maping.get(row1[q])
            q += 1
    return processed_data 

In [5]:
def knn_algorithm(processed_train_data, processed_test_data,K):
    
    # Defining empty lists 
    data_without_labels = []
    train_data_class_labels = []
    test_data_class_labels = []
    estimated_class_labels = []
    
    # Populating these lists.
    for item in processed_train_data:
        temp20 = item[0:-1]
        data_without_labels.append(temp20)
    for item in processed_train_data:
        temp21 = item[-1]
        train_data_class_labels.append(temp21)
    for item in processed_test_data:
        temp23 = item[-1]
        test_data_class_labels.append(temp23)
    
    # Calculating standard deviation and mean of the dataset
    data_without_labels_stdev = numpy.std(data_without_labels,axis=0)
    data_without_labels_mean = numpy.mean(data_without_labels, axis=0)

    # Normalizing the original data
    temp30 = 0
    while temp30 < len(data_without_labels):
        temp_normal = ((data_without_labels[temp30]-data_without_labels_mean)/data_without_labels_stdev).tolist()
        temp_normal.append(train_data_class_labels[temp30])
        data_without_labels[temp30] = temp_normal
        temp30 += 1
    processed_normalized_data = data_without_labels
    
    temp25 = 0
    while temp25 < len(processed_test_data):
        test_instance = processed_test_data[temp25]
        temp31 = test_instance[0:len(test_instance)-1]
        tdp_normalized = (temp31 - data_without_labels_mean)/data_without_labels_stdev
        distance1=[]
        k_near_neighbor=[]
        
        # Finding distances between test records and training records
        temp32 = 0
        while temp32 < len(processed_normalized_data):
            distance1.append((processed_normalized_data[temp32],(distance.euclidean(processed_normalized_data[temp32][0:len(processed_normalized_data[0])-1],tdp_normalized))))
            temp32 += 1
        
        # Sorting according to distance    
        distance1.sort(key=operator.itemgetter(1))
        
        # Populating the k_near_neighbor list
        temp33 = 0
        while temp33 < K:
            k_near_neighbor.append(distance1[temp33][0])
            temp33 = temp33 + 1
        
        
        # Finding the class label for the test record based on the labels of the k nearest labels.
    
        k_near_labels = []
        for item in k_near_neighbor:
            temp51 = item[len(item)-1]
            k_near_labels.append(temp51)
        
        k_near_mapping = dict()
        temp50 = 0
        while temp50 < len(k_near_labels):
            if k_near_labels[temp50] in k_near_mapping:
                k_near_mapping[k_near_labels[temp50]] += 1
            else:
                k_near_mapping[k_near_labels[temp50]] = 1
            temp50 = temp50 + 1    
        max_value = (max(k_near_mapping.values()))
        
        final_class = []
        for key,value in k_near_mapping.items():
            if value == max_value:
                temp52 = (key)
                final_class.append(temp52)    
        found_labels = final_class
        estimated_class_labels.append(found_labels)
        temp25 += 1
        
#     print(estimated_class_labels)
    
    # Passing the test calss labesl and te estimated class labels to arpf_performance function to calculate Accuracy, Precision, Recall and f-measure
    A, P, R, FM= aprf_performance(test_data_class_labels, estimated_class_labels)
    return A, P, R, FM


In [6]:
# Run this code if u want to provide training and testing data which is in the same file.
# Taking input from the user for the value K
while True:
    try:
        K = int(input("Enter the number of neighbor(K) for the KNN Algorithm\t"))
        
    # In case of a error(such as entering a string insterd of number) the except block will be executed
    except:
        print("Error: Please enter a number(int)")
        continue
        
    # This block will be executed after correct value is entered
    else:
        print("K is set to be {}\n".format(K))
        break
        
# Taking input from the user for the total number of cross validation folds
while True:
    try:
        cv_fold = int(input("Enter the number of cross validation folds\t"))
        
    # In case of a error(such as entering a string insterd of number) the except block will be executed
    except:
        print("Error: Please enter a number(int)")
        continue
        
    # This block will be executed after correct value is entered
    else:
        print("Number of cress-validation folds is set to be {}\n".format(K))
        print("----------------------------------------------------------------------------------------\n")
        break

# K = 55
# cv_fold = 10

# Enter the data file name, open it and assign it to the variable data_file
while True:
    try:
        my_file = input("Enter the name of the data file:")
        
    # In case of a error the except block will be executed
    except:
        print("Error: Could not find file or read data")
        continue
        
    # This block will be executed after correct data file name is entered
    else:
        print("Correct data file found and imported\n")
        print("----------------------------------------------------------------------------------------\n")
        break
        
my_file = "project3_dataset2.txt"

# Preprocessing the data
processed_data = data_read_process(my_file)

# Passing the processed data into KNN algorithm
print(f"The KNN results for K = {K}, folds = {cv_fold} and dataset = {my_file} are shown below:")
knn_cvfold(cv_fold,processed_data,K)

The KNN results for K = 55, folds = 10 and dataset = project3_dataset2.txt are shown below:
Accuracy = 0.7117839607201311
Precision = 0.6452669552669553
Recall = 0.32792678812415654
F-measure = 0.42421707268766096


In [7]:
# Run this code in case you want to provide different train and test data files
# Taking input for K.
while True:
    try:
        K = int(input("Enter the number of neighbor(K) for the KNN Algorithm\t"))
        
    # In case of a error(such as entering a string insterd of number) the except block will be executed
    except:
        print("Error: Please enter a number(int)")
        continue
        
    # This block will be executed after correct value is entered
    else:
        print("K is set to be {}\n".format(K))
        break


while True:
    try:
        my_train_file = input("Enter the name of the training data file:")
        
    # In case of a error the except block will be executed
    except:
        print("Error: Could not find file or read data")
        continue
        
    # This block will be executed after correct data file name is entered
    else:
        print("Correct data file found and imported\n")
        print("----------------------------------------------------------------------------------------\n")
        break
        
        
while True:
    try:
        my_test_file = input("Enter the name of the testing data file:")
        
    # In case of a error the except block will be executed
    except:
        print("Error: Could not find file or read data")
        continue
        
    # This block will be executed after correct data file name is entered
    else:
        print("Correct data file found and imported\n")
        print("----------------------------------------------------------------------------------------\n")
        break
        
# Preprocessing the train and test files
processed_training_dataset = data_read_process(my_train_file)
processed_test_dataset = data_read_process(my_test_file)

# Running the KNN algorithm and printing the results
A, P, R, FM = knn_algorithm(processed_training_dataset,processed_test_dataset,K)
# Printing the results
print(f"The results for K = {K} and datasets {my_train_file} and {my_test_file} are shown below:\n")
print(f"Accuracy = {A}")
print(f"Precision = {P}")
print(f"Recall = {R}")
print(f"F-measure = {FM}")

Enter the number of neighbor(K) for the KNN Algorithm	10
K is set to be 10

Enter the name of the training data file:project3_dataset3_train.txt
Correct data file found and imported

----------------------------------------------------------------------------------------

Enter the name of the testing data file:project3_dataset3_test.txt
Correct data file found and imported

----------------------------------------------------------------------------------------

The results for K = 10 and datasets project3_dataset3_train.txt and project3_dataset3_test.txt are shown below:

Accuracy = 0.95
Precision = 1.0
Recall = 0.9
F-measure = 0.9473684210526315
